In [1]:
import dadi
import ipywidgets as widgets
from ipywidgets import FloatSlider, interact_manual, interact, HBox, Tab, Button, ButtonStyle, Layout

from IPython.display import display
from Models import get_dadi_model_func, get_dadi_model_params
import pickle

In [3]:
# Upload data
upload = widgets.FileUpload(
               accept='',  # Accepted file extension e.g. '.fs'
               multiple=False  # True to accept multiple files upload else False
           )

In [4]:
#Select grid size
a = widgets.BoundedIntText(
    value=40,
    min=1,
    max=10000,
    step=1,
    description='Grid-x:',
    disabled=False
)
b = widgets.BoundedIntText(
    value=50,
    min=1,
    max=10000,
    step=1,
    description='Grid-y:',
    disabled=False
)
c = widgets.BoundedIntText(
    value=60,
    min=1,
    max=10000,
    step=1,
    description='Grid-z:',
    disabled=False
)

grid_ui = widgets.HBox([a, b, c])


In [5]:
#Select demographic models
model = widgets.Dropdown(
    options=['bottlegrowth_1d', 'growth_1d', 'snm_1d', 'three_epoch_1d', 'two_epoch_1d',
             'bottlegrowth_2d', 'bottlegrowth_split', 'bottlegrowth_split_mig', 'IM', 'IM_pre', 'snm_2d', 'split_mig', 'split_asym_mig'],
    description='Model:',
)


Upload frequency spectrum data, select grid size, select demographic model

In [6]:
# layout widgets
tab = widgets.Tab()
tab_contents = ['File upload', 'Grid points', 'Model']
tab.children = [upload, grid_ui, model]
for title, (index, _) in zip(tab_contents, enumerate(tab.children)):
    tab.set_title(index, title)
tab

In [7]:
# Load the data from user input
uploaded_filename = next(iter(upload.value))
content = upload.value[uploaded_filename]['content']
with open(uploaded_filename, 'wb') as f: f.write(content)
# Read data into dadi
data = dadi.Spectrum.from_file(uploaded_filename)
ns = data.sample_sizes

# These are the grid point settings will use for extrapolation.
pts_l = [a.value, b.value, c.value]

func = get_dadi_model_func(model.value)
# Make the extrapolating version of our demographic model function.
func_ex = dadi.Numerics.make_extrap_log_func(func)

Select upper and lower bounds

In [8]:
params = get_dadi_model_params(model.value)

# upper bounds sliders
upper_sliders = []
for p in params:
    if p == 's':
        s = widgets.BoundedFloatText(
            value=0.5,
            min=0,
            max=1,
            step=0.1,
            description=p+':',
        )
    else:
        s = widgets.BoundedFloatText(
                value=10,
                min=0,
                max=10000,
                step=0.1,
                description=p+':',
        )
    upper_sliders.append(s)
    
upper_ui = widgets.VBox(upper_sliders)

# lower bounds sliders
lower_sliders = []
for p in params:
    if p == 's':
        s = widgets.BoundedFloatText(
            value=0.5,
            min=0,
            max=1,
            step=0.1,
            description=p+':',
        )
    else:
        s = widgets.BoundedFloatText(
                value=0,
                min=0,
                max=10000,
                step=0.1,
                description=p+':',
        )
    lower_sliders.append(s)
    
lower_ui = widgets.VBox(lower_sliders)

# layout upper and lower bouunds
header  = Button(description='Lower bounds',
                 layout=Layout(width='auto', grid_area='header'),
                 style=ButtonStyle(button_color='lightblue'))
lower_ui = widgets.VBox([header, lower_ui])

header  = Button(description='Upper bounds',
                 layout=Layout(width='auto', grid_area='header'),
                 style=ButtonStyle(button_color='lightblue'))
upper_ui = widgets.VBox([header, upper_ui])


widgets.HBox([lower_ui, upper_ui])


In [9]:
lower_bound = []
for s in lower_sliders:
    lower_bound.append(s.value)

upper_bound = []
for s in upper_sliders:
    upper_bound.append(s.value)


Interactive plots to choose initial p0

In [10]:
def plot_func_2d(**sliders_dict):    
    p0 = [x for x in sliders_dict.values()]
    print('Here are your model ('+model.value+') parameters:')
    print(p0)
    test_model = func_ex(p0, ns, pts_l)
    dadi.Plotting.plot_2d_comp_multinom(test_model, data, vmin=1, resid_range=3, show=True)

def plot_func_1d(**sliders_dict):
    p0 = [x for x in sliders_dict.values()]
    print('Here are your model ('+model.value+') parameters:')
    print(p0)
    test_model = func_ex(p0, ns, pts_l)
    dadi.Plotting.plot_1d_comp_multinom(test_model, data, vmin=1, resid_range=3, show=True)

    
sliders_dict = {}
for p, l, u in zip(params, lower_bound, upper_bound):
    sliders_dict[p+':'] = FloatSlider(
        value=0.5,
        min=l,
        max=u,
        step=0.1,
    )

if '1d' not in model.value:
    interact_manual(plot_func_2d, **sliders_dict)
else:
    interact_manual(plot_func_1d, **sliders_dict)
    

interactive(children=(FloatSlider(value=0.5, description='nu1:', max=10.0), FloatSlider(value=0.5, description…

In [11]:
p0 = []
for p in params:
    p0.append(sliders_dict[p+':'].value)
dic = {
    'params':p0,
    'demographic_model':func,
    'sample_sizes':ns,
    'gird_points':pts_l,
    'lower_bound':lower_bound,
    'upper_bound':upper_bound,
}
def save_params(Filename):
    print("Saving demographic model information as: "+Filename)
    fid=open(Filename,'wb')
    pickle.dump(dic, fid, 2)
    fid.close()
print('What file name do you want to save you demographic model information under?')
interact_manual(save_params, Filename='demo_params.bpkl')

What file name do you want to save you demographic model information under?


interactive(children=(Text(value='demo_params.bpkl', description='Filename'), Button(description='Run Interact…

<function __main__.save_params(Filename)>

{'params': [1.7, 1.1, 0.9, 1.6], 'demographic_model': <function split_mig at 0x11f271c80>, 'sample_sizes': array([20, 20]), 'gird_points': [40, 50, 60], 'lower_bound': [0.0, 0.0, 0.0, 0.0], 'upper_bound': [10.0, 10.0, 10.0, 10.0]}
